<a href="https://colab.research.google.com/github/SLVmain/PyTorch_start/blob/Les_03/GB_HW_03_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
import torch.nn.functional as F
from torch import nn
from torch import optim

import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm_notebook
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!ls drive/MyDrive/'Colab Notebooks'

'pytorch"Июльский_курс_(2020)_PRO_Занятие_25_Pytorch_Часть_1.ipynb"'
 train.csv
 База_знаний_УИИ_и_разборы
 Полезности
 стажировка


In [10]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
data.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [11]:
data.columns

Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'activation_date',
       'user_type', 'image', 'image_top_1', 'deal_probability'],
      dtype='object')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 18 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   item_id               1503424 non-null  object 
 1   user_id               1503424 non-null  object 
 2   region                1503424 non-null  object 
 3   city                  1503424 non-null  object 
 4   parent_category_name  1503424 non-null  object 
 5   category_name         1503424 non-null  object 
 6   param_1               1441848 non-null  object 
 7   param_2               848882 non-null   object 
 8   param_3               640859 non-null   object 
 9   title                 1503424 non-null  object 
 10  description           1387148 non-null  object 
 11  price                 1418062 non-null  float64
 12  item_seq_number       1503424 non-null  int64  
 13  activation_date       1503424 non-null  object 
 14  user_type             1503424 non-

In [18]:
numeric_feature_names = ['price', 'item_seq_number', 'image_top_1', 'deal_probability']
numeric_features = data[numeric_feature_names]
numeric_features.head()

,price,item_seq_number,image_top_1,deal_probability
0,400.0,2,1008.0,0.12789
1,3000.0,19,692.0,0.00000
2,4000.0,9,3032.0,0.43177
3,2200.0,286,796.0,0.80323
4,40000.0,3,2264.0,0.20797


In [19]:
for col in numeric_features.columns:
  if numeric_features[col].isna().sum() > 0:
    numeric_features[col].fillna(numeric_features[col].median(), inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [20]:
numeric_features.isna().sum()

price               0
item_seq_number     0
image_top_1         0
deal_probability    0
dtype: int64

In [23]:
scaler = StandardScaler()

numeric_features[['price']] = scaler.fit_transform(numeric_features[['price']])
numeric_features[['item_seq_number']] = scaler.fit_transform(numeric_features[['item_seq_number']])
numeric_features[['image_top_1']] = scaler.fit_transform(numeric_features[['image_top_1']])

numeric_features.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,price,item_seq_number,image_top_1,deal_probability
0,-0.004593,-0.133095,-0.235462,0.12789
1,-0.004553,-0.130044,-0.573543,0.00000
2,-0.004538,-0.131839,1.929969,0.43177
3,-0.004566,-0.082131,-0.462276,0.80323
4,-0.003984,-0.132915,1.108304,0.20797


In [24]:
numeric_features.iloc[:, :-1]

,price,item_seq_number,image_top_1
0,-0.004593,-0.133095,-0.235462
1,-0.004553,-0.130044,-0.573543
2,-0.004538,-0.131839,1.929969
3,-0.004566,-0.082131,-0.462276
4,-0.003984,-0.132915,1.108304
...,...,...,...
1503419,-0.004595,-0.132198,-1.219748
1503420,-0.004596,-0.133274,-0.039674
1503421,-0.004579,-0.124481,-0.027906
1503422,-0.004594,-0.127532,-1.220818


In [25]:
numeric_features.iloc[:, -1:]

,deal_probability
0,0.12789
1,0.00000
2,0.43177
3,0.80323
4,0.20797
...,...
1503419,0.25019
1503420,0.60000
1503421,0.39569
1503422,0.00000


In [26]:
train, test = train_test_split(numeric_features, test_size=0.25, random_state=13)

In [27]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df.copy()

        self.X = torch.Tensor(self.df.iloc[:, :-1].values)
        self.y = np.array(self.df.iloc[:, -1:].values.tolist())
    
    def __len__(self):
        return self.df.shape[0]


    def __getitem__(self, index):
        return (self.X[index], self.y[index])


In [28]:
x_train = MyDataset(train)
x_test = MyDataset(test)

In [38]:
train_loader = torch.utils.data.DataLoader(x_train, batch_size=256, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(x_test, batch_size=256, shuffle=True, num_workers=2)

In [48]:
for batch, label in train_loader:
    print(batch.shape)
    print(label.shape)
    print(label[:5])
    break

torch.Size([256, 3])
torch.Size([256, 1])
tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.4217]], dtype=torch.float64)


In [49]:
#F.tanh(x), F.leaky_relu(x, 0.05), F.relu(x)
# гипотеза: сделать стандартизацию слоем BatchNorm1d


class SigmoidNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, 4 * hidden_dim)
        self.batch1 = nn.BatchNorm1d(4 * hidden_dim)
        self.dp = nn.Dropout(0.20)
        self.layer2 = nn.Linear(4 * hidden_dim, 2 * hidden_dim)
        self.batch2 = nn.BatchNorm1d(2 * hidden_dim)
        self.layer3 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.layer4 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        
        x = self.layer1(x)
        x = F.leaky_relu(x, 0.05)
        x = self.batch1(x)
        x = self.dp(x)
        x = self.layer2(x)
        x = F.leaky_relu(x, 0.05)
        x = self.batch2(x)
        x = self.dp(x)
        x = self.layer3(x)
        x = F.leaky_relu(x, 0.05)
        
        x = self.layer4(x)
        x = torch.sigmoid(x)
        return x




In [32]:
def rmseloss(y_pred, y_true, eps=1e-6):
  loss = torch.sqrt(torch.mean((torch.log(y_pred+eps)-torch.log(y_true+eps))**2))
  return loss

In [50]:
net = SigmoidNN(3, 50)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [34]:
from tqdm import tqdm

In [51]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = rmseloss(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.005
[1,   301] loss: 1.020
[1,   601] loss: 0.846
[1,   901] loss: 0.843
[1,  1201] loss: 0.844
[1,  1501] loss: 0.844
[1,  1801] loss: 0.841
[1,  2101] loss: 0.841
[1,  2401] loss: 0.839
[1,  2701] loss: 0.839
[1,  3001] loss: 0.836
[1,  3301] loss: 0.839
[1,  3601] loss: 0.839
[1,  3901] loss: 0.833
[1,  4201] loss: 0.836


 10%|█         | 1/10 [00:23<03:34, 23.87s/it]

[2,     1] loss: 0.003
[2,   301] loss: 0.835
[2,   601] loss: 0.836
[2,   901] loss: 0.834
[2,  1201] loss: 0.832
[2,  1501] loss: 0.835
[2,  1801] loss: 0.834
[2,  2101] loss: 0.834
[2,  2401] loss: 0.834
[2,  2701] loss: 0.835
[2,  3001] loss: 0.834
[2,  3301] loss: 0.837
[2,  3601] loss: 0.831
[2,  3901] loss: 0.832
[2,  4201] loss: 0.833


 20%|██        | 2/10 [00:50<03:22, 25.27s/it]

...

[9,     1] loss: 0.003
[9,   301] loss: 0.827
[9,   601] loss: 0.826
[9,   901] loss: 0.830
[9,  1201] loss: 0.828
[9,  1501] loss: 0.826
[9,  1801] loss: 0.829
[9,  2101] loss: 0.829
[9,  2401] loss: 0.827
[9,  2701] loss: 0.827
[9,  3001] loss: 0.827
[9,  3301] loss: 0.826
[9,  3601] loss: 0.824
[9,  3901] loss: 0.825
[9,  4201] loss: 0.829


 90%|█████████ | 9/10 [03:37<00:23, 23.88s/it]

[10,     1] loss: 0.003
[10,   301] loss: 0.826
[10,   601] loss: 0.827
[10,   901] loss: 0.826
[10,  1201] loss: 0.827
[10,  1501] loss: 0.827
[10,  1801] loss: 0.826
[10,  2101] loss: 0.829
[10,  2401] loss: 0.827
[10,  2701] loss: 0.825
[10,  3001] loss: 0.826
[10,  3301] loss: 0.827
[10,  3601] loss: 0.824
[10,  3901] loss: 0.829
[10,  4201] loss: 0.830


100%|██████████| 10/10 [04:00<00:00, 24.09s/it]

Training is finished!


In [52]:
net2 = SigmoidNN(3, 50) #чтобы сравнить оптимизаторы, инициализируем сеть заново
optimizer = torch.optim.Adam(net2.parameters(), lr=0.0004)

In [53]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net2(inputs)
        loss = rmseloss(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.005
[1,   301] loss: 1.019
[1,   601] loss: 0.846
[1,   901] loss: 0.844
[1,  1201] loss: 0.840
[1,  1501] loss: 0.841
[1,  1801] loss: 0.837
[1,  2101] loss: 0.836
[1,  2401] loss: 0.838
[1,  2701] loss: 0.839
[1,  3001] loss: 0.837
[1,  3301] loss: 0.833
[1,  3601] loss: 0.834
[1,  3901] loss: 0.834
[1,  4201] loss: 0.834


 10%|█         | 1/10 [00:25<03:50, 25.57s/it]

[2,     1] loss: 0.003
[2,   301] loss: 0.834
[2,   601] loss: 0.834
[2,   901] loss: 0.830
[2,  1201] loss: 0.834
[2,  1501] loss: 0.832
[2,  1801] loss: 0.832
[2,  2101] loss: 0.832
[2,  2401] loss: 0.831
[2,  2701] loss: 0.831
[2,  3001] loss: 0.831
[2,  3301] loss: 0.832
[2,  3601] loss: 0.831
[2,  3901] loss: 0.830
[2,  4201] loss: 0.826


 20%|██        | 2/10 [00:50<03:23, 25.41s/it]

...

[9,     1] loss: 0.003
[9,   301] loss: 0.822
[9,   601] loss: 0.824
[9,   901] loss: 0.818
[9,  1201] loss: 0.821
[9,  1501] loss: 0.821
[9,  1801] loss: 0.822
[9,  2101] loss: 0.818
[9,  2401] loss: 0.822
[9,  2701] loss: 0.822
[9,  3001] loss: 0.819
[9,  3301] loss: 0.821
[9,  3601] loss: 0.821
[9,  3901] loss: 0.823
[9,  4201] loss: 0.821


 90%|█████████ | 9/10 [03:46<00:25, 25.08s/it]

[10,     1] loss: 0.003
[10,   301] loss: 0.819
[10,   601] loss: 0.821
[10,   901] loss: 0.820
[10,  1201] loss: 0.821
[10,  1501] loss: 0.823
[10,  1801] loss: 0.820
[10,  2101] loss: 0.822
[10,  2401] loss: 0.822
[10,  2701] loss: 0.820
[10,  3001] loss: 0.820
[10,  3301] loss: 0.822
[10,  3601] loss: 0.820
[10,  3901] loss: 0.819
[10,  4201] loss: 0.820


100%|██████████| 10/10 [04:11<00:00, 25.16s/it]

Training is finished!


In [54]:
net3 = SigmoidNN(3, 50) #чтобы сравнить оптимизаторы, инициализируем сеть заново
optimizer = torch.optim.RMSprop(net3.parameters())

for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net3(inputs)
        loss = rmseloss(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.005
[1,   301] loss: 0.863
[1,   601] loss: 0.846
[1,   901] loss: 0.843
[1,  1201] loss: 0.839
[1,  1501] loss: 0.837
[1,  1801] loss: 0.836
[1,  2101] loss: 0.835
[1,  2401] loss: 0.834
[1,  2701] loss: 0.834
[1,  3001] loss: 0.832
[1,  3301] loss: 0.833
[1,  3601] loss: 0.831
[1,  3901] loss: 0.831
[1,  4201] loss: 0.831


 10%|█         | 1/10 [00:24<03:37, 24.18s/it]

[2,     1] loss: 0.003
[2,   301] loss: 0.830
[2,   601] loss: 0.830
[2,   901] loss: 0.829
[2,  1201] loss: 0.829
[2,  1501] loss: 0.830
[2,  1801] loss: 0.829
[2,  2101] loss: 0.827
[2,  2401] loss: 0.830
[2,  2701] loss: 0.826
[2,  3001] loss: 0.830
[2,  3301] loss: 0.825
[2,  3601] loss: 0.827
[2,  3901] loss: 0.826
[2,  4201] loss: 0.827


 20%|██        | 2/10 [00:48<03:14, 24.25s/it]

...

[9,     1] loss: 0.003
[9,   301] loss: 0.823
[9,   601] loss: 0.823
[9,   901] loss: 0.823
[9,  1201] loss: 0.822
[9,  1501] loss: 0.823
[9,  1801] loss: 0.822
[9,  2101] loss: 0.824
[9,  2401] loss: 0.822
[9,  2701] loss: 0.822
[9,  3001] loss: 0.822
[9,  3301] loss: 0.823
[9,  3601] loss: 0.823
[9,  3901] loss: 0.823
[9,  4201] loss: 0.822


 90%|█████████ | 9/10 [03:36<00:23, 23.89s/it]

[10,     1] loss: 0.003
[10,   301] loss: 0.821
[10,   601] loss: 0.820
[10,   901] loss: 0.822
[10,  1201] loss: 0.823
[10,  1501] loss: 0.822
[10,  1801] loss: 0.821
[10,  2101] loss: 0.820
[10,  2401] loss: 0.823
[10,  2701] loss: 0.822
[10,  3001] loss: 0.822
[10,  3301] loss: 0.823
[10,  3601] loss: 0.825
[10,  3901] loss: 0.823
[10,  4201] loss: 0.822


100%|██████████| 10/10 [04:00<00:00, 24.00s/it]

Training is finished!


# На 10 эпохах немного лучше сходимость у ADAM, затем RMSprop, хуже всего - у SDG

In [ ]:
"""class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        return loss"""

In [ ]:
#train = pd.read_csv('../train.csv.zip',compression='zip',parse_dates=['activation_date'])
#test = pd.read_csv('../test.csv.zip',compression='zip',parse_dates=['activation_date'])

#x = x.view(x.shape[0], -1)